In [1]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import scipy as sp

import sys
sys.path.insert(0, '../../')
import DLDMD as dl
import LossDLDMD as lf
import Data as dat
import Training as tr

%matplotlib inline

In [2]:
print(tf.__version__)

2.5.0


In [3]:
def edmd(x, num_pred):
    x = tf.transpose(x, perm=[0, 2, 1])
    x_m = x[:, :, :-1]
    x_p = x[:, :, 1:]
    
    S, U, V = tf.linalg.svd(x_m, compute_uv=True, full_matrices=False)
    sm = np.max(S)
    r = S.shape[-1]
    Sri = tf.linalg.diag(1./S[:, :r])
    Ur = U[:, :, :r]
    Urh = tf.linalg.adjoint(Ur)
    Vr = V[:, :, :r]
    
    kmat = x_p @ Vr @ Sri @ Urh
    evals, evecs = tf.linalg.eig(kmat)
    phim = tf.linalg.solve(evecs, tf.cast(x_m, dtype=tf.complex128))
    x0 = phim[:, :, 0]
    x0 = x0[:, :, tf.newaxis]
    
    pred = tf.TensorArray(tf.complex128, size=num_pred)
    pred = pred.write(0, evecs @ x0)
    evals_iter = tf.identity(evals)
    for ii in range(num_pred):
        tmp = evecs @ tf.linalg.diag(evals_iter) @ x0
        pred = pred.write(ii, tmp)
        evals_iter = evals_iter * evals
    pred = tf.transpose(tf.squeeze(pred.stack()), perm=[1, 2, 0])
    return phim, evals, evecs, pred

# Setup

In [4]:
# Figure parameters
plot_save_path = './analysis_results/'
font = {'family': 'DejaVu Sans', 'size': 18}
matplotlib.rc('font', **font)
fontsize = 18
figsize = (15, 10)
dpisave = 300

# Initialize the compute device
DEVICE = '/GPU:0'
GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for gpu in GPUS:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    DEVICE = '/CPU:0'
    
tf.keras.backend.set_floatx('float64')  # !! Set precision for the entire model here
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))
print("Num GPUs available: {}".format(len(GPUS)))
print("Training at precision: {}".format(tf.keras.backend.floatx()))
print("Training on device: {}".format(DEVICE))

TensorFlow version: 2.5.0
Eager execution: True
Num GPUs available: 0
Training at precision: float64
Training on device: /CPU:0


# Load model and data

In [5]:
# SET THIS PATH (w/o file extension). Both '.pkl' and '.h5' files should have same name
model_path = './trained_models/pendulum_2022-04-15-0024/epoch_100_loss_-2.33'
hyp_params_path = model_path + '.pkl'
weight_path = model_path + '.h5'

# Load the hyper parameters
hyp_params = pickle.load(open(hyp_params_path, 'rb'))

# Set Tensorflow backend precision
tf.keras.backend.set_floatx(hyp_params['precision'])
print("Using precision: {}\n".format(tf.keras.backend.floatx()))

# Create evenly spaced data
double_time = hyp_params['time_final']*2
num_steps = int(double_time / hyp_params['delta_t'])
num_ic = 10
x1 = np.linspace(-3.00001, 0.1, num_ic)
x2 = np.zeros(num_ic)
data_mat = np.zeros((num_ic, 2, num_steps), dtype=np.float64)
for ii in range(num_ic):
    data_mat[ii, :, 0] = np.array([x1[ii], x2[ii]], dtype=np.float64)
    for jj in range(num_steps - 1):
        data_mat[ii, :, jj+1] = dat.rk4(data_mat[ii, :, jj],
                                        hyp_params['delta_t'],
                                        dat.dyn_sys_pendulum)
test_data = np.transpose(data_mat, [0, 2, 1])

# Load evenly spaced rings for test trajectories
# test_data = pickle.load(open('evenly_spaced_trajs.pkl', 'rb'))
# test_data = tf.cast(test_data, dtype=hyp_params['precision'])
print("Test data shape: {}".format(test_data.shape))
print(hyp_params)

Using precision: float64

Test data shape: (10, 800, 2)
{'sim_start': '2022-04-15-0024', 'experiment': 'pendulum', 'plot_path': './training_results/pendulum_2022-04-15-0024', 'model_path': './trained_models/pendulum_2022-04-15-0024', 'device': '/CPU:0', 'precision': 'float64', 'num_init_conds': 15000, 'num_train_init_conds': 10000, 'num_val_init_conds': 3000, 'num_test_init_conds': 2000, 'time_final': 20, 'delta_t': 0.05, 'num_time_steps': 401, 'num_pred_steps': 401, 'max_epochs': 100, 'save_every': 100, 'plot_every': 1, 'num_observables': 1, 'threshhold': 6, 'observation_dimension': 0, 'optimizer': 'adam', 'batch_size': 64, 'phys_dim': 2, 'latent_dim': 2, 'hidden_activation': <function relu at 0x7f914310ad40>, 'bias_initializer': <class 'keras.initializers.initializers_v2.Zeros'>, 'num_en_layers': 3, 'num_en_neurons': 128, 'kernel_init_enc': <keras.initializers.initializers_v2.TruncatedNormal object at 0x7f9142ee90d0>, 'kernel_init_dec': <keras.initializers.initializers_v2.TruncatedNo

In [15]:
# Fix hyper parameters for running the model on test data
hyp_params['pretrain'] = False
hyp_params['batch_size'] = test_data.shape[0]
hyp_params['num_time_steps'] = test_data.shape[1]
hyp_params['latent_dim'] = test_data.shape[2]
hyp_params['phys_dim'] = test_data.shape[2]
# hyp_params['num_observables'] = 70

print(hyp_params)

# Load the trained DLDMD model weights
model = dl.DLDMD(hyp_params)
model.num_pred_steps = model.num_time_steps
model.time_final = int(model.num_time_steps*model.delta_t)
model(test_data)
model.load_weights(weight_path)

# Initialize the loss function
loss = lf.LossDLDMD(hyp_params)
print("Number of prediction steps: ", model.num_pred_steps)
num_obsvs_opt = int(180/2)
num_time_steps = int(hyp_params['num_time_steps'])
model.num_observables = num_obsvs_opt
model.window = num_time_steps - (num_obsvs_opt - 1)
loss.num_observables = num_obsvs_opt
loss.window = num_time_steps - (num_obsvs_opt - 1)

print(vars(model))

{'sim_start': '2022-04-15-0024', 'experiment': 'pendulum', 'plot_path': './training_results/pendulum_2022-04-15-0024', 'model_path': './trained_models/pendulum_2022-04-15-0024', 'device': '/CPU:0', 'precision': 'float64', 'num_init_conds': 15000, 'num_train_init_conds': 10000, 'num_val_init_conds': 3000, 'num_test_init_conds': 2000, 'time_final': 20, 'delta_t': 0.05, 'num_time_steps': 800, 'num_pred_steps': 401, 'max_epochs': 100, 'save_every': 100, 'plot_every': 1, 'num_observables': 1, 'threshhold': 6, 'observation_dimension': 0, 'optimizer': 'adam', 'batch_size': 10, 'phys_dim': 2, 'latent_dim': 2, 'hidden_activation': <function relu at 0x7f914310ad40>, 'bias_initializer': <class 'keras.initializers.initializers_v2.Zeros'>, 'num_en_layers': 3, 'num_en_neurons': 128, 'kernel_init_enc': <keras.initializers.initializers_v2.TruncatedNormal object at 0x7f9142ee90d0>, 'kernel_init_dec': <keras.initializers.initializers_v2.TruncatedNormal object at 0x7f9142ee93d0>, 'ae_output_activation': 

In [17]:
def hankel_dmd(Y):
    Y = np.transpose(Y, [0, 2, 1])
    print("shape of Y: ", np.shape(Y))
    nobs = 90
    winsize = model.num_time_steps - (nobs - 1)
    # Perform DMD method.  Note, we need to be careful about how we break the concatenated Hankel matrix apart.

    gm = tf.Variable(tf.zeros([nobs*hyp_params['phys_dim'], hyp_params['batch_size'] * (winsize - 1)], dtype=hyp_params['precision']))
    gp = tf.Variable(tf.zeros([nobs*hyp_params['phys_dim'], hyp_params['batch_size'] * (winsize - 1)], dtype=hyp_params['precision']))

    print("gm shape: ", gm.shape)
    print("gp shape: ", gp.shape)
    for jj in range(hyp_params['phys_dim']):
        Yobserved = (tf.squeeze(Y[:, jj, :])).numpy()
        for ll in range(hyp_params['batch_size']):
            tseries = Yobserved[ll, :]
            tcol = tseries[:nobs]
            trow = tseries[(nobs - 1):]
            hmat = np.flipud(sp.linalg.toeplitz(tcol[::-1], trow))

            print("hmat shape: ", np.shape(hmat))
            print("nobs: ", nobs)
            print("winsize: ", winsize)
            gm[jj*nobs:(jj+1)*nobs, ll * (winsize - 1):(ll + 1) * (winsize - 1)].assign(hmat[:, :-1])
            gp[jj*nobs:(jj+1)*nobs, ll * (winsize - 1):(ll + 1) * (winsize - 1)].assign(hmat[:, 1:])

    sig, U, V = tf.linalg.svd(gm, compute_uv=True, full_matrices=False)
    sig_inv = tf.linalg.diag(1.0 / sig)
    Uh = tf.linalg.adjoint(U)
    gpV = gp @ V
    A = gpV @ sig_inv @ Uh
    evals, evecs = tf.linalg.eig(A)
    phi = tf.linalg.solve(evecs, tf.cast(gm, dtype=tf.complex128))

    normgp = tf.norm(gp, ord='fro', axis=[-2, -1])
    normgpV = tf.norm(gpV, ord='fro', axis=[-2, -1])
    dmdloss = tf.math.sqrt(normgp**2. - normgpV**2.)/tf.math.sqrt(tf.cast(hyp_params['batch_size']*(winsize-1), dtype=hyp_params['precision']))

    # Build reconstruction
    phiinit = phi[:, ::(winsize-1)]
    initconds = tf.cast(tf.transpose(tf.squeeze(Y[:, :, 0])), dtype=tf.complex128)
    sigp, Up, Vp = tf.linalg.svd(phiinit, compute_uv=True, full_matrices=False)
    sigp_inv = tf.cast(tf.linalg.diag(1.0 / sigp), dtype=tf.complex128)
    kmat = initconds @ Vp @ sigp_inv @ tf.linalg.adjoint(Up)
    recon = tf.reshape(tf.transpose(tf.math.real(kmat @ phi)), [hyp_params['batch_size'], winsize-1, hyp_params['phys_dim']])
    # comment out dmdloss from return
    return recon, evals, evecs, phi, dmdloss

# Run the DLDMD model

In [18]:
with tf.device(DEVICE):
    [y, x_ae, x_adv, y_adv, weights, evals, evecs, phi, dmdloss] = model(test_data, training=False)
    losses = loss([y, x_ae, x_adv, y_adv, weights, evals, evecs, phi, dmdloss], test_data)

print("Loss: {loss:2.7f}".format(loss=losses.numpy()))
print("Log10 Loss: {loss:2.7f}".format(loss=np.log10(losses.numpy())))

Loss: 0.6605270
Log10 Loss: -0.1801094


In [19]:
[x_pred, evals, evecs, phim, dmdloss] = hankel_dmd(test_data)
print("shape of xpred: ", np.shape(x_pred))
print("shape of xadv: ", np.shape(x_adv))

shape of Y:  (10, 2, 800)
gm shape:  (180, 7100)
gp shape:  (180, 7100)
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
winsize:  711
hmat shape:  (90, 711)
nobs:  90
wi

InvalidArgumentError: Eigen decomposition was not successful. The input might not be valid. [Op:Eig]

# Run standard DMD

In [ ]:
# EDMD on the unencoded data
# [phim, evals, evecs, x_pred] = edmd(test_data, num_pred=test_data.shape[1])
# x_pred = np.real(tf.transpose(x_pred, perm=[0, 2, 1]))

# Visualize results

In [ ]:
fs = 20
ts = 20
lw = 2.0
ms = 20.0
figsize = (12, 12)
skip = 1

# DLHDMD reconstruction
fig = plt.figure(1, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], color='red', linestyle='solid', lw=lw)
    plt.plot(x_adv[ii, :, 0], x_adv[ii, :, 1], color='blue', linestyle='dotted', ms=ms)
plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], color='red', linestyle='solid', lw=lw, label='Test data')
plt.plot(x_adv[ii, 0, 0], x_adv[ii, 0, 1], color='blue', linestyle='dotted', ms=20*ms, label='DLHDMD')
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.legend(fontsize=fs, loc='upper right')
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)
# plt.savefig("reconstruction_pendulum.png")

# DMD reconstruction
fig = plt.figure(2, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], color='red', linestyle='solid', lw=lw)
    plt.plot(x_pred[ii, :, 0], x_pred[ii, :, 1], color='blue', linestyle='dotted', ms=ms)
plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], color='red', linestyle='solid', lw=lw, label='Test data')
plt.plot(x_pred[ii, 0, 0], x_pred[ii, 0, 1], color='blue', linestyle='dotted', ms=20*ms, label='Hankel DMD')
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.legend(fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

# Plot the trajectories in phase space and latent space
fig = plt.figure(3, figsize=figsize)
for ii in range(0, test_data.shape[0], skip):
    plt.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'k', linestyle='solid', lw=lw)
plt.xlabel(r'$x$', fontsize=fs)
plt.ylabel(r'$\dot{x}$', fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)

# Plot the trajectories in latent space
fig = plt.figure(4, figsize=figsize)
for ii in range(y_adv.shape[0]):
    plt.plot(y[ii, :, 0], y[ii, :, 1], 'k', linestyle='solid', ms=ms)
plt.xlabel(r'$\tilde{x}_{1}$', fontsize=fs)
plt.ylabel(r'$\tilde{x}_{2}$', fontsize=fs)
plt.axis('equal')
ax = plt.gca()
ax.tick_params(axis='both', which='major', labelsize=ts)
ax.tick_params(axis='both', which='minor', labelsize=ts)
# plt.savefig("latent_pendulum.png")

plt.show()